### Training the veracity prediction model

#### Steps
- Load the reference DBpedia graph
- Load the TransE embedding model
- Load train and test knowledge graphs
- Perform feature engineering
    - Convert entities and relations to tensors
    - Compute features

- Split training data into train and validation sets
- Choose a model and train
    - Logistic regression
    - RandomForest
    - Ensemble

- Make predictions on Test data
- Save output file as 'result.ttl'


In [ ]:
# !pip install pykeen rdflib torch --quiet

In [41]:
# !unzip /content/trans-e-embeddings.zip -d trans-e-embeddings

In [42]:
import torch
import numpy as np
from rdflib import Graph, URIRef, RDF
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

In [ ]:
REFERENCE_FILE = "data/dbpedia-reference-kg.nt"
reference_knowledge_graph = Graph()
reference_knowledge_graph.parse(REFERENCE_FILE)

<Graph identifier=N4036f036add6428f918e8750fb708ab2 (<class 'rdflib.graph.Graph'>)>

In [ ]:
reference_data_pykeen = TriplesFactory.from_path_binary("trans-e-embeddings/training_triples")


INFO:pykeen.triples.triples_factory:Loading from file:///content/fokg_mini_project/trans-e-embeddings/training_triples


In [45]:
reference_data_pykeen

TriplesFactory(num_entities=897222, num_relations=686, create_inverse_triples=True, num_triples=911628)

In [ ]:
model = torch.load("trans-e-embeddings/trained_model.pkl", map_location=torch.device('cpu'), weights_only=False)


In [47]:
reference_triples = []
for head, rel, tail in reference_knowledge_graph.triples((None, None, None)):
    if isinstance(head, URIRef) and isinstance(tail, URIRef):
        reference_triples.append((str(head), str(rel), str(tail)))

# triple lookup map
triple_lookup = set(reference_triples)

entity_to_id = reference_data_pykeen.entity_to_id
relation_to_id = reference_data_pykeen.relation_to_id

def get_embeddings(subj, pred, obj):
    emb_dim = 120
    if subj not in entity_to_id or obj not in entity_to_id or pred not in relation_to_id:
        return np.zeros(emb_dim), np.zeros(emb_dim), np.zeros(emb_dim)

    s_id = entity_to_id[subj]
    p_id = relation_to_id[pred]
    o_id = entity_to_id[obj]

    with torch.no_grad():
        s_emb = model.entity_representations[0](indices=torch.tensor([s_id]))
        p_emb = model.relation_representations[0](indices=torch.tensor([p_id]))
        o_emb = model.entity_representations[0](indices=torch.tensor([o_id]))

    return (s_emb[0].cpu().numpy(),
            p_emb[0].cpu().numpy(),
            o_emb[0].cpu().numpy())


def build_feature_vector(h, r, t):
    h_emb, r_emb, t_emb = get_embeddings(h, r, t)
    eps = 1e-8

    # Vector norms
    h_n = np.linalg.norm(h_emb) + eps
    r_n = np.linalg.norm(r_emb) + eps
    t_n = np.linalg.norm(t_emb) + eps

    # Normalized vectors
    h_u = h_emb / h_n
    r_u = r_emb / r_n
    t_u = t_emb / t_n

    features = []

    # TransE translation errors
    translation = h_emb + r_emb - t_emb
    features.append(np.linalg.norm(translation, ord=1))
    features.append(np.linalg.norm(translation, ord=2))
    features.append(np.linalg.norm(translation, ord=np.inf))

    # Head–tail distances
    features.append(np.linalg.norm(h_emb - t_emb, ord=1))
    features.append(np.linalg.norm(h_emb - t_emb, ord=2))

    # Cosine similarities
    features.append(np.dot(h_u, t_u))
    features.append(np.dot(h_u, r_u))
    features.append(np.dot(r_u, t_u))

    # Magnitude statistics
    features.extend([
        np.log(h_n), np.log(r_n), np.log(t_n),
        h_n, r_n, t_n
    ])

    # Mean absolute values
    features.extend([
        np.mean(np.abs(h_emb)),
        np.mean(np.abs(r_emb)),
        np.mean(np.abs(t_emb)),
    ])

    # Maximum absolute values
    features.extend([
        np.max(np.abs(h_emb)),
        np.max(np.abs(r_emb)),
        np.max(np.abs(t_emb)),
    ])

    # Dot products
    features.append(np.dot(h_emb, r_emb))
    features.append(np.dot(r_emb, t_emb))
    features.append(np.dot(h_emb, t_emb))

    # Higher-order interactions
    features.append(np.sum(h_emb * r_emb * t_emb))
    features.append(np.sum((h_emb + r_emb) * t_emb))

    # Structural indicators
    features.append(1.0 if (h, r, t) in triple_lookup else 0.0)
    features.append(1.0 if (t, r, h) in triple_lookup else 0.0)

    # Approximate degree features
    h_deg = sum(1 for (x, _, _) in reference_triples if x == h)
    t_deg = sum(1 for (_, _, y) in reference_triples if y == t)

    features.append(np.log(h_deg + 1))
    features.append(np.log(t_deg + 1))

    return np.asarray(features, dtype=np.float32)




In [ ]:
from sklearn.model_selection import train_test_split

X_train = []
y_train = []
X_test = []

training_graph = Graph()
training_graph.parse("data/KG-2022-train.nt.txt")

from urllib.parse import unquote

def clean_uri(u: str) -> str:
    return unquote(u.strip("<>"))

for statement in training_graph.subjects(RDF.type, RDF.Statement):

    subject = training_graph.value(statement, RDF.subject)
    predicate = training_graph.value(statement, RDF.predicate)
    obj = training_graph.value(statement, RDF.object)
    val = training_graph.value(statement, URIRef("http://swc2017.aksw.org/hasTruthValue"))


    s = clean_uri(subject.n3().strip("<>"))
    p = clean_uri(predicate.n3().strip("<>"))
    o = clean_uri(obj.n3().strip("<>"))

    if (
        s not in reference_data_pykeen.entity_to_id
        or o not in reference_data_pykeen.entity_to_id
        or p not in reference_data_pykeen.relation_to_id
    ):
        continue

    X_train.append(build_feature_vector(s, p, o) )
    # y_train.append(val)
    y_train.append(float(val))



X_train = np.array(X_train)
y_train = np.array(y_train)


X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, shuffle=True
)

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42,
    shuffle=True,
)

In [49]:
X_train_np.shape

(884, 29)

In [50]:
reference_triples

[('http://dbpedia.org/resource/Charles_Henry_Cooper',
  'http://dbpedia.org/ontology/country',
  'http://dbpedia.org/resource/England'),
 ('http://dbpedia.org/resource/Daniel_Calveti',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Caracas'),
 ('http://dbpedia.org/resource/Gatewen_Marsh',
  'http://dbpedia.org/ontology/location',
  'http://dbpedia.org/resource/United_Kingdom'),
 ('http://dbpedia.org/resource/Lila_Clunas',
  'http://dbpedia.org/ontology/birthPlace',
  'http://dbpedia.org/resource/Glasgow'),
 ('http://dbpedia.org/resource/1520_Sedgwick_Avenue',
  'http://dbpedia.org/ontology/location',
  'http://dbpedia.org/resource/The_Bronx'),
 ('http://dbpedia.org/resource/Consort_Mountain',
  'http://dbpedia.org/ontology/locatedInArea',
  'http://dbpedia.org/resource/Alberta'),
 ('http://dbpedia.org/resource/Haruto_Dam',
  'http://dbpedia.org/ontology/country',
  'http://dbpedia.org/resource/Japan'),
 ('http://dbpedia.org/resource/Madeleine_Meilleur',
  'h

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score


rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

lr = LogisticRegression(
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)


rf.fit(X_train_np, y_train_np)
lr.fit(X_train_np, y_train_np)


rf_probs = rf.predict_proba(X_test_np)[:, 1]
rf_auc = roc_auc_score(y_test_np, rf_probs)
rf_pred = rf.predict(X_test_np)
rf_acc = accuracy_score(y_test_np, rf_pred)

print(f"Random Forest ROC AUC: {rf_auc:.4f}")
print(f"Random Forest Accuracy: {rf_acc:.4f}")

lr_probs = lr.predict_proba(X_test_np)[:, 1]
lr_auc = roc_auc_score(y_test_np, lr_probs)
lr_pred = lr.predict(X_test_np)
lr_acc = accuracy_score(y_test_np, lr_pred)

print(f"Logistic Regression ROC AUC: {lr_auc:.4f}")
print(f"Logistic Regression Accuracy: {lr_acc:.4f}")


ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('lr', lr)
    ],
    voting='soft'   # IMPORTANT for ROC AUC
)

ensemble.fit(X_train_np, y_train_np)

ens_probs = ensemble.predict_proba(X_test_np)[:, 1]
ens_auc = roc_auc_score(y_test_np, ens_probs)
ens_pred = ensemble.predict(X_test_np)
ens_acc = accuracy_score(y_test_np, ens_pred)

print(f"Ensemble ROC AUC: {ens_auc:.4f}")
print(f"Ensemble Accuracy: {ens_acc:.4f}")


Random Forest ROC AUC: 0.8251
Random Forest Accuracy: 0.7387
Logistic Regression ROC AUC: 0.8772
Logistic Regression Accuracy: 0.7613
Ensemble ROC AUC: 0.8630
Ensemble Accuracy: 0.7793


In [ ]:
testing_graph = Graph()
testing_graph.parse("data/KG-2022-test.nt.txt")

X_test = []
test_triple_iris = []



for statement in testing_graph.subjects(RDF.type, RDF.Statement):

    subject = testing_graph.value(statement, RDF.subject)
    predicate = testing_graph.value(statement, RDF.predicate)
    obj = testing_graph.value(statement, RDF.object)
    # val = training_graph.value(statement, URIRef("http://swc2017.aksw.org/hasTruthValue"))


    s = clean_uri(subject.n3().strip("<>"))
    p = clean_uri(predicate.n3().strip("<>"))
    o = clean_uri(obj.n3().strip("<>"))

    if (
        s not in reference_data_pykeen.entity_to_id
        or o not in reference_data_pykeen.entity_to_id
        or p not in reference_data_pykeen.relation_to_id
    ):
        # print(s,p ,o)
        continue

    X_test.append(build_feature_vector(s, p, o) )
    test_triple_iris.append(statement)




In [54]:
test_pred = lr.predict(X_test)


In [ ]:
# save predictions to file

X_test_np = np.array(X_test)
test_pred = ensemble.predict_proba(X_test_np)[:, 1]  
test_pred_as_list = test_pred.tolist()

result_file = open('result.ttl', 'w')
for i in range(len(test_pred_as_list)):
    print(f"<{test_triple_iris[i]}> <http://swc2017.aksw.org/hasTruthValue> \"{test_pred_as_list[i]}\"^^<http://www.w3.org/2001/XMLSchema#double> .", file=result_file)
result_file.close()

In [ ]:
# from google.colab import files
# files.download('result.ttl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>